<a href="https://colab.research.google.com/github/TalpsG/Note/blob/master/%E7%AC%AC%E5%85%AD%E7%AB%A0%E5%AD%A6%E4%B9%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

互相关操作

In [ ]:
import torch
from torch import nn
def corr2d(x,k):
    h,w = k.shape
    y = torch.zeros(x.shape[0]-h + 1,x.shape[1]-w+1)
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            y[i,j] = (x[i:i+h,j:j+w]*k).sum()
    return y

In [ ]:
x = torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
k = torch.tensor([[0.0,1.0],[2.0,3.0]])
corr2d(x,k)

tensor([[19., 25.],
        [37., 43.]])

卷积层,就是对输入和卷积核进行运算的层。卷积层中的参数是卷积核的权重和偏移。

In [ ]:
class Conv2d(nn.Module):
    def __init__(self,kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self , x):
        return corr2d(x,self.weight)+self.bias



边缘检测

In [ ]:
x = torch.ones((6,8))
x[:,2:6] = 0
k = torch.tensor([[1.0,-1.0]])


In [ ]:
y = corr2d(x,k)
y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

复杂数值的卷积核不可能依靠人类手动设计，需要通过学习来自动生成卷积核。

In [ ]:
# 构建一个二维卷积层
conv2d = nn.Conv2d(1,1,kernel_size=(1,2),bias = False)

x = x.reshape(1,1,6,8)
y = y.reshape(1,1,6,7)

lr = 3e-2

for i in range(10):
    y_hat = conv2d(x)
    l = (y_hat-y)**2
    conv2d.zero_grad()
    l.sum().backward()
    # l是从y_hat来的，backward的可以让模型参数计算梯度
    conv2d.weight.data[:] -=lr*conv2d.weight.grad
    if (i+1)%2 == 0:
        print(f'epoch {i+1} , loss{l.sum():.3f}')

conv2d.weight.data

epoch 2 , loss11.640
epoch 4 , loss2.724
epoch 6 , loss0.773
epoch 8 , loss0.259
epoch 10 , loss0.096


tensor([[[[ 1.0158, -0.9539]]]])

### 6.2.6  feature map 和receptive field
featuremap 就是经过卷积层的输出。
receptive field就是卷积核的过滤的区域，这个区域的信息实际上包含了不止卷积核大小的信息。



### 作业
1. 对角线边缘的图像
    - 卷积核k作用于该图像的结果
    - 转置后再卷积的结果

In [32]:
x = torch.ones((6,6))
k = torch.tensor([[1.0,-1.0]])
k = k.transpose()

TypeError: transpose() received an invalid combination of arguments - got (), but expected one of:
 * (int dim0, int dim1)
 * (name dim0, name dim1)


In [ ]:
y = corr2d(x,k)
y

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

## 6.3 padding and stride
多层卷积时常常会丢失边缘像素(个人理解是经过卷积后边缘的像素和其他像素混合了，故而丢失了).
我们可以使用padding,对边缘来填充像素，来使得卷积后边缘的像素仍然可以被保留下来。
由于一般卷积后输出会比原图小，因此我们可以在图像外部填充几圈，使得输入和输出的大小一致。

In [18]:
import torch
from torch import nn

def comp_conv2d(conv2d,x):
    x = x.reshape((1,1)+x.shape)
    y = conv2d(x)
    return y.reshape(y.shape[2:])

conv2d = nn.Conv2d(1,1,kernel_size=3,padding=1)
conv2d.weight.data = conv2d.weight.data - conv2d.weight.data

x = torch.rand(size=(8,8))
y = comp_conv2d(conv2d,x)
conv2d.bias.data

tensor([-0.1498])

### 6.3.2 stride
stride其实就是卷积核平移的像素数，平移的远会导致输出结果变得更小，当然运算量也会更小。


In [ ]:
conv2d= nn.Conv2d(1,1,kernel_size=3,padding=1,stride=2)
comp_conv2d(conv2d,x).shape

torch.Size([4, 4])

In [ ]:
conv2d = nn.Conv2d(1,1,kernel_size=(3,5),padding=(0,1),stride=(3,4))
comp_conv2d(conv2d,x).shape

torch.Size([2, 2])

## 多输入多输出
### 多输入通道
当输入有多个通道时，我们的卷积核也要有多个通道。
卷积核对每个通道做互相关运算后，得到的结果可以求和得到二维张量。


In [4]:
import torch
from torch import nn
def corr2d(x,k):
  h,w = k.shape
  y = torch.zeros((x.shape[0]-h+1,x.shape[1]-w+1))
  for i in range(y.shape[0]):
    for j in range(y.shape[1]):
      y[i,j]= (x[i:i+h,j:j+w]*k).sum()

  return y
def corr2d_multi_in(x,k):
  return sum(corr2d(x1,k1) for x1,k1 in zip(x,k))

In [30]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

### 多输出通道
在神经网络架构中，随着神经网络层数的加深，常常会增加输出通道，因为输出通道数增多可以帮助网络捕获更多复杂和抽象的特征。随着通道数增多，为了维持运算效率，常常会降低各个通道的分辨率。

In [27]:
def corr2d_multi_in_out(x,k):
  return torch.stack([corr2d_multi_in(x,k1) for k1 in k ],0)

In [31]:
K = torch.stack((K,K+1,K+2),0)
corr2d_multi_in_out(X,K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

多输出通道的卷积层实际上就是将对每一个通道的卷积结果`stack`到了一起。

### 1x1卷积核
1x1卷积核虽然不能改变形状，但是我们可以改变输出的通道数。

In [33]:
def corr2d_multi_in_out_1x1(X,K):
  c_i,h,w = X.shape
  c_o = K.shape[0]
  X = X.reshape((c_i,h*w))
  K = K.reshape(c_o,c_i)
  y = torch.matmul(K,X)
  return y.reshape(c_o,h,w)


In [35]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6


### 小结
多输入多输出通道可以扩展卷积层模型。

1x1的卷积层相当于全连接层。

## 池化层
目的是为了降低卷积层对位置的敏感性

In [37]:
import torch
from torch import nn
def pool2d(X,pool_size,mode='max'):
  h,w = pool_size
  y = torch.zeros((x.shape[0]-h+1,x.shape[1]-w+1))
  for i in range(y.shape[0]):
    for j in range(y.shape[1]):
      if mode == 'max':
        Y[i,j] = X[i:i+h,j:j+w].max()
      else:
        Y[i,j] = X[i:i+h,j:j+w].mean()
  return y